In [1]:
import requests as req
import pandas as pd
import json
import time

# Utils

## Consts

In [3]:
GITHUB_REST_URL = 'https://api.github.com/'
TOKEN = ''
HEADERS = {
  'Authorization': f'bearer {TOKEN}',
}

# Functions

In [21]:
def measure_rest(path: str):
    rest_endpoint = GITHUB_REST_URL + path
    start_time = time.time()
    response = req.get(rest_endpoint, headers=HEADERS)
    end_time = time.time()
    if response.status_code == 200:
        return end_time - start_time, len(response.content)
    
    raise Exception(f'Erro ao fazer requisição: {response.status_code} \n {response.text}\n {path}')


def super_measure_rest(owner: str, repo: str):
    start_time = time.time()
    response = req.get(f'{GITHUB_REST_URL}repos/{owner}/{repo}/pulls?state=all&per_page=5', headers=HEADERS)
    time2, size = get_pull_request_details(owner, repo, response)
    end_time = time.time()
    if response.status_code == 200:
        return end_time - start_time + time2, size + len(response.content)
    
    raise Exception(f'Erro ao fazer requisição: {response.status_code} \n {response.text}\n ')



def get_pull_request_details(owner, repo, pull_requests):
    time_maluco_2, time_maluco_3, time_maluco_4, time_maluco_5, time_maluco_6 = 0, 0, 0, 0, 0
    size_maluco_2, size_maluco_3, size_maluco_4, size_maluco_5, size_maluco_6 = 0, 0, 0, 0, 0
    owner = 'Netflix'
    repo = 'Hystrix'
    if pull_requests:
        pull_requests = pull_requests.json()
        for pr in pull_requests:
            pr_number = pr['number']
            time_maluco_2, size_maluco_2 = measure_rest(f'repos/{owner}/{repo}/pulls/{pr_number}')
            time_maluco_3, size_maluco_3 = measure_rest(f'repos/{owner}/{repo}/pulls/{pr_number}/commits')
            time_maluco_4, size_maluco_4 = measure_rest(f'repos/{owner}/{repo}/pulls/{pr_number}/files')
            time_maluco_5, size_maluco_5 = measure_rest(f'repos/{owner}/{repo}/issues/{pr_number}/comments')
            time_maluco_6, size_maluco_6 = measure_rest(f'repos/{owner}/{repo}/pulls/{pr_number}/reviews')

    
    return (time_maluco_2 + time_maluco_3 + time_maluco_4 + time_maluco_5 + time_maluco_6), (size_maluco_2 + size_maluco_3 + size_maluco_4 + size_maluco_5 + size_maluco_6)

## Paths

In [5]:
path_1 = 'users/Netflix/repos?per_page=5'
path_2 = 'repos/Netflix/Hystrix/issues?state=open&per_page=5'
path_3 = 'repos/Netflix/Hystrix/pulls?per_page=5'

# Experiment

In [22]:
data = []
for _ in range(10):
    rest_time, rest_size = measure_rest(path_1)
    data.append({
        'type_query': '1',
        'rest_time': rest_time,
        'rest_size': rest_size,
    })

for _ in range(10):
    rest_time, rest_size = measure_rest(path_2)
    data.append({
        'type_query': '2',
        'rest_time': rest_time,
        'rest_size': rest_size,
    })

for _ in range(10):
    rest_time, rest_size = measure_rest(path_3)
    data.append({
        'type_query': '3',
        'rest_time': rest_time,
        'rest_size': rest_size,
    })

for _ in range(10):
    rest_time, rest_size = super_measure_rest('Netflix', 'Hystrix')
    data.append({
        'type_query': '4',
        'rest_time': rest_time,
        'rest_size': rest_size,
    })
    
df = pd.DataFrame(data)
df.to_csv('api_rest_results.csv', index=False)